In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import pickle
from tqdm import tqdm
import re
import json
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F
from collections import Counter
import json

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device('cuda:0')

## Prepare

- **Config & Data**

In [ ]:
from data import *
with open('config.json') as f:
    config = json.load(f)

In [ ]:
config['train']['sample_num'] = 16
config['train']['save_frequency'] = 1000
config['train']['type'] = 1
config

In [ ]:
sources = np.load('../../data2/sources.npy')
target_inputs = np.load('../../data2/target_inputs.npy')
target_outputs = np.load('../../data2/target_outputs.npy')
embedding_matrix = np.load('../../data2/embedding_matrix2.npy')
with open('../../data2/dict.pkl', 'rb') as f:
    news_index,category_dict,word_dict = pickle.load(f)
index2word = {}
for k,v in word_dict.items():
    index2word[v] = k
print(len(word_dict),embedding_matrix.shape)

- **Model**

In [ ]:
from pensmodule.Generator import HeadlineGen
model = HeadlineGen(config['model'], embedding_matrix, index2word, device, pointer_gen=True).to(device)

In [ ]:
from pensmodule.UserEncoder import NRMS

usermodel = NRMS(embedding_matrix)
usermodel.load_state_dict(torch.load('../../runs/userencoder/NAML-2.pkl'))
usermodel = usermodel.to(device)
usermodel.eval()

- **Load Trainer**

In [ ]:
from pensmodule.Generator.train import *
trainer = Trainer(config, model, usermodel, device, experiment_name='test2')

## Pretrain Seq2seq model

In [ ]:
global_user_embed = np.load('../../data2/global_user_embed2.npy')
global_user_embed = torch.as_tensor(global_user_embed, device=device).float()

In [ ]:
trainer._init_optimizer()

In [ ]:
for epoch in range(1,6):
    print('epoch:', epoch)
    s_dset = Seq2SeqDataset(sources, target_inputs, target_outputs)
    train_iter = DataLoader(s_dset, batch_size=128, shuffle=True)
    
    trainer.pretrain(train_iter, global_user_embed)
    if epoch%5 == 0:
        trainer.save_checkpoint(tag='pretrain_epoch_'+str(epoch))

## Train Personalized Generator

- **data loader**

In [ ]:
with open('../../data2/TrainUsers.pkl', 'rb') as f:
    TrainUsers = pickle.load(f)
with open('../../data2/TrainSamples.pkl', 'rb') as f:
    TrainSamples = pickle.load(f)

In [ ]:
news_scoring = np.load('../../data2/news_scoring2.npy')

In [ ]:
i_dset = ImpressionDataset(news_scoring, sources, target_inputs, target_outputs, TrainUsers, TrainSamples)
data_loader = DataLoader(i_dset, batch_size=16, shuffle=True)

- **train**

In [ ]:
trainer.load_checkpoint(tag='pretrain_epoch_5')

In [ ]:
trainer._init_evaluator_()

In [ ]:
trainer._init_context_()

In [ ]:
optimizer_params={'lr': 0.00001}
scheduler_params={'step_size': 200, 'gamma': 0.98}
trainer._init_optimizer(optimizer_params=optimizer_params,scheduler_params=scheduler_params)

In [ ]:
# trainer.train(data_loader,train_option='ppo')

In [ ]:
trainer.train(data_loader,train_option='a2c')

## Test

In [ ]:
with open('../../data2/TestUsers.pkl', 'rb') as f:
    TestUsers = pickle.load(f)
with open('../../data2/TestSamples.pkl', 'rb') as f:
    TestSamples = pickle.load(f)

In [ ]:
from pensmodule.Generator import *
model_path = '../../runs/seq2seq/test2/checkpoint_train_a2c_step_300.pth'
model = load_model_from_ckpt(model_path).to(device)
model.eval()

In [ ]:
from pensmodule.UserEncoder import NRMS

usermodel = NRMS(embedding_matrix)
usermodel.load_state_dict(torch.load('../../runs/userencoder/NAML-2.pkl'))
usermodel = usermodel.to(device)
usermodel.eval()

In [ ]:
i_dset = TestImpressionDataset(news_scoring, sources, TestUsers, TestSamples)
test_iter = DataLoader(i_dset, batch_size=16, shuffle=True)

In [ ]:
scores1, scores2, scoresf = predict(usermodel, model, test_iter, device, index2word, beam=False, beam_size=5, eos_id=2)

In [ ]:
scores1.mean(), scores2.mean(), scoresf.mean()